In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
from akorn import *
from flh import *
%load_ext autoreload
%autoreload 2

%matplotlib qt

In [3]:
%load_ext autoreload
%autoreload 2
# import numpy as np
# import math
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr, data



utils = importr('utils')
base = importr('base')
glmgen = importr('glmgen')
%matplotlib qt

std = 0.3
var = std**2

# mse_akorn = []
# mse_tf = []



def doppler(x, epsilon):
    return np.sin(2 * np.pi * (1 + epsilon) / (x + epsilon))

from util import piecewise_linear

print("Done!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done!


In [8]:
mse_akorn = []
mse_addle = []
mse_tf = []
mse_tf_dof = []
# Ts = [100, 5000, 100_000, 200_000, 500_000]
Ts = [5000, 10_000, 20_000, 40_000]
# Ts = [50, 100]
# 


# # Ts = [100]
runs_per_T = 1

from util import *

akorn_fv = None
dof_fv = None
tf_fv = None
    
for T in Ts:
    print("T = ", T)
    X = np.linspace(0, T, T)/T 
    Xr = robjects.FloatVector(X)

    k = 5
    G = np.array([0]*(T-k) + [1 + j for j in range(k)])

    mses_T_tf = []
    mses_T_tf_dof = []
    mses_T_akorn = []

    for l in range(runs_per_T):
        print("Run number " + str(l))
        Y = (G + np.random.normal(0, std, T))

        ak = AKORN(X, Y, var)
        ak.train()
        fv = ak.preds
        


        mses_T_akorn.append(((fv - G)**2).mean())


        Yr = robjects.FloatVector(Y)
        tf = glmgen.trendfilter(Xr, Yr, k = 1)
        rcode1 = "predict(%s)" %(tf.r_repr())
        tf_fit = np.array(robjects.r(rcode1)).T

        best_mse = float('inf')
        best_mse_est = float('inf')
        best_mse_dof = float('inf')
        best_tf_fit = None
        best_tf_fit_dof = None
        for t in tf_fit:
            tf_mse = ((t - G)**2).mean() #Oracle risk
            tf_mse_est = (np.sum(((t - Y)**2)) + 2*var*count_linear_pieces(t)) #Stein risk estimate
            if tf_mse < best_mse:
                best_mse = tf_mse
                best_tf_fit = t
            if tf_mse_est < best_mse_est:
                best_mse_est = tf_mse_est
                best_mse_dof = tf_mse
                best_tf_fit_dof = t
                
        mses_T_tf.append(((best_tf_fit - G)**2).mean())
        mses_T_tf_dof.append(((best_tf_fit_dof - G)**2).mean())
        
        akorn_fv = fv
        tf_dof_fv = best_tf_fit_dof
        tf_fv = best_tf_fit

    #     Z_tf.append(best_tf_fit)
    mse_tf.append(mses_T_tf)
    mse_tf_dof.append(mses_T_tf_dof)
    mse_akorn.append(mses_T_akorn)
    

avg_mses_akorn = [np.array(mse_T).mean() for mse_T in mse_akorn]
avg_mses_tf = [np.array(mse_T).mean() for mse_T in mse_tf]
avg_mses_tf_dof = [np.array(mse_T).mean() for mse_T in mse_tf_dof]
print("Done!")

T =  5000
Run number 0
lr =  0.004929778922540199
run
T =  10000
Run number 0
lr =  0.005162012291578354
run
T =  20000
Run number 0
lr =  0.005498900444077486
run
T =  40000
Run number 0
lr =  0.00437800528537834
run
Done!


In [ ]:
from sklearn.linear_model import LinearRegression
akorn_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(avg_mses_akorn))
tf_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(np.array(avg_mses_tf)))
tf_dof_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(np.array(avg_mses_tf_dof)))

In [6]:
def quick_round_str(z):
    return str(round(z, 5))

plt.clf()
from util import subplots_centered
T = Ts[-1]
X = np.linspace(0, T, T)/T 

k = 5
G = np.array([0]*(T-k) + [1 + j for j in range(k)])
f, ax = plt.subplots(1, 2)
# f, ax = subplots_
ax0 = ax[0]
ax1 = ax[1]
ax1.plot(range(T), G, color = "black")
ax1.plot(range(T), tf_fv, color = "tab:blue")
# ax1.plot(range(T), tf_dof_fv, color = "turquoise")

ax1.plot(range(T), Y, alpha = 0.15, color = "black")
ax1.plot(range(T), akorn_fv, color = "tab:orange")
# ax1.plot(range(T), addle_fv, color = "green")

ax1.yaxis.set_visible(False)

# f.suptitle("Rates for PW Linear Function")
ax0.set_xlabel("n")
ax0.set_ylabel("MSE")

ax0.set_ylim([10**(-3), 10**(-1)])
# ax1.set_ylim([10**(-4), 10**(0)])

ax0.grid(True)


ax0.loglog(Ts, avg_mses_akorn, marker = 'o', label = "akorn", color = "tab:orange")
ax0.loglog(Ts, avg_mses_tf, marker = 'o', label = "oracle trend filtering", color = "tab:blue")
# ax0.loglog(Ts, avg_mses_addle, marker = 'o', label = "addle, rate = " + quick_round_str(addle_ref.coef_[0]), color = "green")
ax0.loglog(Ts, avg_mses_tf_dof, marker = 'o', label = "tf dof", color = "blue")

# f.legend()